In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import urllib
import zipfile

tf.__version__

'2.10.0'

In [3]:
def download_and_extract_data():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/certificate/household_power.zip'
    urllib.request.urlretrieve(url, 'household_power.zip')
    with zipfile.ZipFile('household_power.zip', 'r') as zip_ref:
        zip_ref.extractall()

In [4]:
download_and_extract_data()

In [5]:
df = pd.read_csv('household_power_consumption.csv', sep=',',
                     infer_datetime_format=True, index_col='datetime', header=0)

In [6]:
N_FEATURES = len(df.columns)

In [7]:
data = df.values
data[:10]

array([[  4.216,   0.418, 234.84 ,  18.4  ,   0.   ,   1.   ,  17.   ],
       [  5.36 ,   0.436, 233.63 ,  23.   ,   0.   ,   1.   ,  16.   ],
       [  5.374,   0.498, 233.29 ,  23.   ,   0.   ,   2.   ,  17.   ],
       [  5.388,   0.502, 233.74 ,  23.   ,   0.   ,   1.   ,  17.   ],
       [  3.666,   0.528, 235.68 ,  15.8  ,   0.   ,   1.   ,  17.   ],
       [  3.52 ,   0.522, 235.02 ,  15.   ,   0.   ,   2.   ,  17.   ],
       [  3.702,   0.52 , 235.09 ,  15.8  ,   0.   ,   1.   ,  17.   ],
       [  3.7  ,   0.52 , 235.22 ,  15.8  ,   0.   ,   1.   ,  17.   ],
       [  3.668,   0.51 , 233.99 ,  15.8  ,   0.   ,   1.   ,  17.   ],
       [  3.662,   0.51 , 233.86 ,  15.8  ,   0.   ,   2.   ,  16.   ]])

In [8]:
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data

In [9]:
data = normalize_series(data, data.min(axis=0), data.max(axis=0))

In [10]:
data[:10]

array([[0.43377912, 0.47826087, 0.04036551, 0.43564356, 0.        ,
        0.01282051, 0.85      ],
       [0.55716135, 0.49885584, 0.0355582 , 0.54950495, 0.        ,
        0.01282051, 0.8       ],
       [0.55867127, 0.56979405, 0.03420739, 0.54950495, 0.        ,
        0.02564103, 0.85      ],
       [0.56018119, 0.57437071, 0.03599523, 0.54950495, 0.        ,
        0.01282051, 0.85      ],
       [0.37446074, 0.60411899, 0.04370282, 0.37128713, 0.        ,
        0.01282051, 0.85      ],
       [0.35871441, 0.597254  , 0.04108065, 0.35148515, 0.        ,
        0.02564103, 0.85      ],
       [0.3783434 , 0.59496568, 0.04135876, 0.37128713, 0.        ,
        0.01282051, 0.85      ],
       [0.3781277 , 0.59496568, 0.04187525, 0.37128713, 0.        ,
        0.01282051, 0.85      ],
       [0.37467645, 0.58352403, 0.03698848, 0.37128713, 0.        ,
        0.01282051, 0.85      ],
       [0.37402934, 0.58352403, 0.03647199, 0.37128713, 0.        ,
        0.02564103, 0.8

In [11]:
SPLIT_TIME = int(len(data) * 0.5)

In [12]:
x_train = data[:SPLIT_TIME]
x_valid = data[SPLIT_TIME:]

In [13]:
x_train.shape, x_valid.shape

((43200, 7), (43200, 7))

In [14]:
BATCH_SIZE = 32
N_PAST = 24
N_FUTURE = 24
SHIFT = 1

In [15]:
def windowed_dataset(series, batch_size, n_past=24, n_future=24, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=n_past + n_future, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.map(lambda w: (w[:n_past], w[n_past:]))
    return ds.batch(batch_size).prefetch(1)

In [16]:
train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)

In [17]:
valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)

In [38]:
tf.random.set_seed(42)

inputs = tf.keras.layers.Input(shape=(N_PAST, N_FEATURES))
x = tf.keras.layers.Conv1D(filters=128, kernel_size=5, padding="causal", activation="relu")(inputs)
x = tf.keras.layers.MaxPool1D()(x)
x = tf.keras.layers.Conv1D(filters=128, kernel_size=5, padding="causal", activation="relu")(x)
x = tf.keras.layers.MaxPool1D()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(N_FUTURE * N_FEATURES)(x)
outputs = tf.keras.layers.Reshape((N_FUTURE, N_FEATURES))(x)
model_1 = tf.keras.Model(inputs = inputs, outputs = outputs, name = "model_1")

In [39]:
model_1.compile(loss = "mae",
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [40]:
model_1.fit(train_set,
            epochs=10,
            validation_data=valid_set,
            callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=100, restore_best_weights=True)])

Epoch 1/10
1349/1349 [==============================] - 6s 4ms/step - loss: 0.0705 - accuracy: 0.6544 - val_loss: 0.0587 - val_accuracy: 0.6572
Epoch 2/10
1349/1349 [==============================] - 6s 4ms/step - loss: 0.0568 - accuracy: 0.7011 - val_loss: 0.0510 - val_accuracy: 0.6831
Epoch 3/10
1349/1349 [==============================] - 6s 5ms/step - loss: 0.0532 - accuracy: 0.7124 - val_loss: 0.0556 - val_accuracy: 0.7001
Epoch 4/10
1349/1349 [==============================] - 6s 5ms/step - loss: 0.0505 - accuracy: 0.7232 - val_loss: 0.0484 - val_accuracy: 0.7049
Epoch 5/10
1349/1349 [==============================] - 6s 5ms/step - loss: 0.0492 - accuracy: 0.7291 - val_loss: 0.0478 - val_accuracy: 0.7063
Epoch 6/10
1349/1349 [==============================] - 7s 5ms/step - loss: 0.0490 - accuracy: 0.7357 - val_loss: 0.0492 - val_accuracy: 0.7064
Epoch 7/10
1349/1349 [==============================] - 7s 5ms/step - loss: 0.0481 - accuracy: 0.7369 - val_loss: 0.0475 - val_accuracy:

## TEST CODE

In [41]:
def mae(y_true, y_pred):
    return np.mean(abs(y_true.ravel() - y_pred.ravel()))

In [42]:
def model_forecast(model, series, window_size, batch_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(batch_size, drop_remainder=True).prefetch(1)
    forecast = model.predict(ds)
    return forecast

In [43]:
rnn_forecast = model_forecast(model_1, data, N_PAST, BATCH_SIZE)

2699/2699 [==============================] - 4s 1ms/step


In [44]:
rnn_forecast = rnn_forecast[SPLIT_TIME - N_PAST:-1, 0, :]

In [45]:
x_valid = x_valid[:rnn_forecast.shape[0]]

In [46]:
result = mae(x_valid, rnn_forecast)
result

0.034627849466299356

## Sequential Model

In [60]:
tf.random.set_seed(42)

model_2 = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=128, kernel_size=5, padding="causal", activation="relu", input_shape=(N_PAST, N_FEATURES)),
    tf.keras.layers.MaxPool1D(),
    tf.keras.layers.Conv1D(filters=128, kernel_size=5, padding="causal", activation="relu"),
    tf.keras.layers.MaxPool1D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(N_FUTURE*N_FEATURES),
    tf.keras.layers.Reshape((N_FUTURE, N_FEATURES))
])

model_2.compile(loss = "mae",
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [61]:
model_2.fit(train_set,
            epochs=10,
            validation_data=valid_set,
            callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=100, restore_best_weights=True)])

Epoch 1/10
1349/1349 [==============================] - 7s 5ms/step - loss: 0.0703 - accuracy: 0.6583 - val_loss: 0.0592 - val_accuracy: 0.6667
Epoch 2/10
1349/1349 [==============================] - 7s 5ms/step - loss: 0.0586 - accuracy: 0.6963 - val_loss: 0.0546 - val_accuracy: 0.6786
Epoch 3/10
1349/1349 [==============================] - 7s 5ms/step - loss: 0.0533 - accuracy: 0.7101 - val_loss: 0.0505 - val_accuracy: 0.6967
Epoch 4/10
1349/1349 [==============================] - 7s 5ms/step - loss: 0.0508 - accuracy: 0.7176 - val_loss: 0.0494 - val_accuracy: 0.7053
Epoch 5/10
1349/1349 [==============================] - 7s 5ms/step - loss: 0.0498 - accuracy: 0.7165 - val_loss: 0.0484 - val_accuracy: 0.7071
Epoch 6/10
1349/1349 [==============================] - 7s 5ms/step - loss: 0.0489 - accuracy: 0.7217 - val_loss: 0.0500 - val_accuracy: 0.7015
Epoch 7/10
1349/1349 [==============================] - 7s 5ms/step - loss: 0.0478 - accuracy: 0.7256 - val_loss: 0.0493 - val_accuracy:

## TEST CODE - 2

In [62]:
rnn_forecast = model_forecast(model_2, data, N_PAST, BATCH_SIZE)

2699/2699 [==============================] - 4s 1ms/step


In [63]:
rnn_forecast = rnn_forecast[SPLIT_TIME - N_PAST:-1, 0, :]

In [64]:
x_valid = x_valid[:rnn_forecast.shape[0]]

In [65]:
result = mae(x_valid, rnn_forecast)
result

0.03489537973609473